In [1]:
import pandas as pd
import boto3
import pyarrow
import awswrangler as wr

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.live import StockDataStream
from alpaca.data.requests import StockBarsRequest, StockTradesRequest
from alpaca.data.timeframe import TimeFrame

from dotenv import load_dotenv
import os
import json
from datetime import datetime, timedelta

In [2]:
load_dotenv()
ALPACA_API_KEY = os.environ.get("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.environ.get("ALPACA_SECRET_KEY")

## Reading Tickers

In [4]:
nasdaq = pd.read_csv("https://s3-alpaca-stock-data.s3.us-west-1.amazonaws.com/tickers/nasdaq100.csv")
nyse = pd.read_csv("https://s3-alpaca-stock-data.s3.us-west-1.amazonaws.com/tickers/nyse100.csv")

In [5]:
nasdaq.head()

,Ticker,Code,Name
0,FOXA,GOOG/NASDAQ_FOXA,21st Century Fox Class A
1,ATVI,GOOG/NASDAQ_ATVI,Activision Blizzard Inc
2,ADBE,GOOG/NASDAQ_ADBE,Adobe Systems Inc
3,AKAM,GOOG/NASDAQ_AKAM,Akamai Technologies Inc
4,ALXN,GOOG/NASDAQ_ALXN,Alexion Pharmaceuticals Inc


In [6]:
nyse.head()

,Ticker,Code,Name
0,MMM,GOOG/NYSE_MMM,3M Co.
1,ABT,GOOG/NYSE_ABT,Abbott Laboratories
2,ACN,GOOG/NYSE_ACN,Accenture PLC Cl A
3,AGN,GOOG/NYSE_AGN,Allergan Inc.
4,MO,GOOG/NYSE_MO,Altria Group Inc.


In [7]:
nasdaq_tickers = list(nasdaq.iloc[:, 0])
nyse_tickers = list(nyse.iloc[:, 0])
tickers = set(nasdaq_tickers + nyse_tickers)

## Batch Data Ingestion

In [8]:
hist_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

In [9]:
def convert_datetime(date: datetime):
    return datetime(date.year, date.month, date.day, 0, 0, 0)

def getStockHistoricalData(client: StockHistoricalDataClient, start_date, end_date = datetime.today()):
    bar_request = StockBarsRequest(
        symbol_or_symbols=tickers,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )
    res = client.get_stock_bars(bar_request)
    return(res)

bars = getStockHistoricalData(hist_client, datetime.today() - timedelta(days = 1)).data

In [33]:
len(bars)

158

In [10]:
BAR_SCHEMA = {
    "symbol": str,
    "high": "float64",
    "low": "float64",
    "open": "float64",
    "timestamp": "datetime64[ns]",
    "trade_count": "float64",
    "volume": "float64",
    "vwap": "float64"
}
df = pd.DataFrame(columns=BAR_SCHEMA.keys()).astype(BAR_SCHEMA)

for i, ticker in enumerate(bars):
    for bar in bars[ticker]:
        entry = {
            "symbol": bar.symbol,
            "high": bar.high,
            "low": bar.low,
            "open": bar.open,
            "timestamp": bar.timestamp,
            "trade_count": bar.trade_count,
            "volume": bar.volume,
            "vwap": bar.vwap
        }
        df.loc[len(df)] = entry


In [11]:
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

In [12]:
df.head()

,symbol,high,low,open,timestamp,trade_count,volume,vwap,year,month,day
0,D,48.5101,47.940,48.25,2023-09-14 04:00:00+00:00,29916.0,2285526.0,48.215720,2023,9,14
1,VOD,10.0200,9.920,9.95,2023-09-14 04:00:00+00:00,12191.0,3263404.0,9.979714,2023,9,14
2,EBAY,44.4450,43.720,43.97,2023-09-14 04:00:00+00:00,33297.0,1948881.0,44.226416,2023,9,14
3,EMC,24.7700,24.641,24.73,2023-09-14 04:00:00+00:00,155.0,85036.0,24.705405,2023,9,14
4,ITW,242.0600,238.270,240.13,2023-09-14 04:00:00+00:00,12191.0,361682.0,239.444986,2023,9,14


## Using AWS SDK (Main Flow)

In [13]:
glue_db_name = "alpaca_stocks_database"
glue_table_name = f"stocks_table_{datetime.now().year}_{datetime.now().month}"

In [14]:
wr.s3.to_parquet(
    df = df,
        path = "s3://s3-alpaca-stock-data/daily/",
        dataset = True,
        partition_cols = ["year", "month", "day"],
        database = glue_db_name,
        table = glue_table_name,
        boto3_session = aws_session,
        mode = "overwrite_partitions"  
)

{'paths': ['s3://s3-alpaca-stock-data/daily/year=2023/month=9/day=14/d46f85b3477f4a53b5d9bc827fb8e3d2.snappy.parquet'],
 'partitions_values': {'s3://s3-alpaca-stock-data/daily/year=2023/month=9/day=14/': ['2023',
   '9',
   '14']}}

In [49]:
df_athena = wr.athena.read_sql_query(f"SELECT * FROM {glue_table_name}", database = glue_db_name)

In [50]:
df_athena

,symbol,high,low,open,timestamp,trade_count,volume,vwap,year,month,day
0,DD,76.42,75.305,75.97,2023-09-11 04:00:00,29616.0,1879465.0,75.757702,2023,9,11
1,FOXA,32.32,31.320,31.36,2023-09-11 04:00:00,46701.0,3229086.0,31.827620,2023,9,11
2,GILD,77.17,76.000,76.05,2023-09-11 04:00:00,60056.0,8053756.0,76.709498,2023,9,11
3,VZ,34.09,33.580,33.70,2023-09-11 04:00:00,111546.0,18883869.0,33.877310,2023,9,11
4,DD,75.98,74.920,75.01,2023-09-12 04:00:00,27848.0,2093238.0,75.389116,2023,9,12
...,...,...,...,...,...,...,...,...,...,...,...
311,AKAM,105.34,103.925,104.20,2023-09-12 04:00:00,29581.0,1321847.0,104.382950,2023,9,12
312,BAX,39.14,38.565,38.93,2023-09-11 04:00:00,30044.0,2790719.0,38.799307,2023,9,11
313,CHKP,136.58,134.850,135.00,2023-09-11 04:00:00,11168.0,512016.0,135.961660,2023,9,11
314,BAX,38.99,38.220,38.94,2023-09-12 04:00:00,26697.0,2121468.0,38.471872,2023,9,12


## Using Boto3

In [124]:
df.to_parquet("stocks.parquet.gzip", engine = "pyarrow", compression = "gzip")

In [125]:
s3 = boto3.client("s3")

file_path = "stocks.parquet.gzip"
bucket_name = "s3-alpaca-stock-data"


In [126]:
s3.upload_file(file_path, bucket_name, "historical/test.parquet.gzip")